In [1]:
import pandas as pd
import numpy as np
import altair as alt

# Set up some global config and variables
alt.renderers.enable('default')
pd.options.mode.chained_assignment = None

def align(data):
    # Find the index with the lowest non-zero starting case number
    base = min(((idx, val) for (idx, val) in enumerate(data) if val[0] > 0), key = lambda x: x[1][0])[0]

    # Align everyone else to this base
    for i in [x for x in range(len(data)) if x != base]:
        padding = min(((x[0], abs(x[1] - data[i][0])) for x in enumerate(data[base])), key = lambda x: x[1])[0]
        data[i] = [0 for i in range(padding)] + data[i]
    return data

# Takes a list of numbers and returns a list with number of rows to double
def doubling(data):
    doubling = []
    for i in reversed(range(len(data))):
        index = -1
        for j in reversed(range(i)):
            if data[i] >= data[j]*2:
                index = j
                break
        doubling.append(i - index if index > 0 else np.nan)
    return list(reversed(doubling))

def streamgraph(df, by, value, sort, limit, stack='zero'):
    zeros = df.groupby('Date', as_index=False).sum()
    zeros = zeros[zeros[value] <= 10].Date
    df = df[~df['Date'].isin(zeros)]
    
    df.loc[df[value]<0, value] = 0
    order = df[df['Date'] == days[-1]].sort_values(sort, ascending=False)
    top = order[by].values.tolist()
    data = df[df[by].isin(top[:limit])]
    others = df[df[by].isin(top[limit:])].groupby('Date', as_index=False).sum()
    others.insert(0, by, 'Others')
    data = data.append(others, ignore_index=True)
    selection = alt.selection_multi(fields=[by], bind='legend')

    base = alt.Chart(data).mark_area().encode(
        alt.X('Date:T', axis=alt.Axis(domain=False, format='%d %b', tickSize=0)),
        alt.Y(value + ':Q', stack=stack, axis=alt.Axis(title=' '.join(value.split('_'))) if stack=='zero' else None),
        alt.Color(by + ':N', scale=alt.Scale(scheme='tableau20'), sort=top[:limit]),
        order=alt.Order('sort:Q'),
        tooltip=alt.Tooltip(['Date:T', by, value]),
        opacity=alt.condition(selection, alt.value(1), alt.value(0.3))
    ).transform_lookup(
        lookup=by,
        from_=alt.LookupData(order, by, [sort]),
        as_=['sort']
    )
    totals = alt.Chart(data.groupby('Date', as_index=False).sum()).mark_line(strokeDash=[6,3]).encode(
        x='Date:T',
        y='rolling_mean:Q',
        color=alt.value('black'),
    ).transform_window(
        rolling_mean='mean(%s)' % value,
        frame=[-7, 7]
    )
    return (base+totals).properties(
        width=1200,
        height=800
    ).add_selection(selection)

def nontimegraph(df, by, value, sort, limit, highlight='', addon=[], xlog=True, ylog=True, rect=None, ascending=False):
    top = set(df[df['Date'] == days[-1]].sort_values(sort, ascending=ascending)[by].values.tolist()[:limit] + addon + [highlight])
    data = df[(df[value] > 1) & (df['Confirmed'] > 100) & (df[by].isin(top))]
    #data.loc[:,value] = data[value].rolling(window=3, win_type='gaussian').mean(std=3)
    selection = alt.selection_multi(fields=[by], bind='legend', init=[{by: highlight}], empty='none', nearest=True)

    # Background
    line = alt.Chart(data).mark_line().encode(
        alt.X('Confirmed:Q', scale=alt.Scale(type='log' if xlog else 'linear'), axis=alt.Axis(title='Cumulative Cases')),
        alt.Y(value + ':Q', scale=alt.Scale(type='log' if ylog else 'linear'), axis=alt.Axis(title=' '.join(value.split('_')))),
        color=alt.Color(by+':N', scale=alt.Scale(scheme='category10' if highlight == '' else 'set2')),
        size=alt.value(1),
        tooltip=alt.Tooltip(['Date:N', by, value])
    ).transform_filter(~selection)
    point = line.mark_circle().encode(
        alt.X('Confirmed:Q', scale=alt.Scale(type='log' if xlog else 'linear')),
        alt.Y(value + ':Q', scale=alt.Scale(type='log' if ylog else 'linear')),
        color=alt.Color(by+':N'),
        size=alt.value(60)
    ).transform_filter(
        alt.datum.Date == days[-1]
    ).transform_filter(~selection)
    text = point.mark_text(
        align='left',
        dx=7,  # Nudges text to right so it doesn't appear on top of the bar
    ).encode(
        text=by+':N',
        size=alt.value(10),
        color=alt.Color(by+':N')
    ).transform_filter(~selection)
    background = line+point+text

    # Foreground
    line = alt.Chart(data).mark_line().encode(
        alt.X('Confirmed:Q', scale=alt.Scale(type='log' if xlog else 'linear'), axis=alt.Axis(title='Cumulative Cases')),
        alt.Y(value + ':Q', scale=alt.Scale(type='log' if ylog else 'linear'), axis=alt.Axis(title=' '.join(value.split('_')))),
        color=alt.value('black'),
        size=alt.value(3),
        tooltip=alt.Tooltip(['Date:N', by, value])
    ).transform_filter(selection)
    point = line.mark_circle().encode(
        alt.X('Confirmed:Q', scale=alt.Scale(type='log' if xlog else 'linear')),
        alt.Y(value + ':Q', scale=alt.Scale(type='log' if ylog else 'linear')),
        color=alt.value('black'),
        size=alt.value(100)
    ).transform_filter(
        alt.datum.Date == days[-1]
    ).transform_filter(selection)
    text = point.mark_text(
        align='left',
        dx=7,  # Nudges text to right so it doesn't appear on top of the bar
    ).encode(
        text=by+':N',
        size=alt.value(15),
        color=alt.value('black')
    ).transform_filter(selection)
    foreground = line+point+text

    start = df[(df[by] == highlight) & (df['Date'] == rect[0])] if rect is not None else pd.DataFrame()
    end = df[(df[by] == highlight) & (df['Date'] == rect[1])] if rect is not None else pd.DataFrame()
    if not start.empty and not end.empty:
        marks = pd.DataFrame([{"start_x": start.Confirmed.values[0], "end_x": end.Confirmed.values[0], "start_y": start[value].values[0], "end_y": end[value].values[0]}])
        rule_rect = alt.Chart(marks).mark_rect(opacity=0.3).encode(x='start_x:Q', x2='end_x:Q')
        rule_point_1 = rule_rect.mark_circle().encode(
            alt.X('start_x:Q', scale=alt.Scale(type='log' if xlog else 'linear')),
            alt.Y('start_y:Q', scale=alt.Scale(type='log' if ylog else 'linear')),
            color=alt.value('grey'),
            size=alt.value(60)
        )
        rule_point_2 = rule_rect.mark_circle().encode(
            alt.X('end_x:Q', scale=alt.Scale(type='log' if xlog else 'linear')),
            alt.Y('end_y:Q', scale=alt.Scale(type='log' if ylog else 'linear')),
            color=alt.value('grey'),
            size=alt.value(60)
        )
        rule_text_1 = rule_point_1.mark_text(
            align='left',
            angle=90,
            dx=10
        ).encode(
            text=alt.value(rect[0]),
            size=alt.value(15),
            color=alt.value('black')
        )
        rule_text_2 = rule_point_2.mark_text(
            align='left',
            angle=90,
            dx=10
        ).encode(
            text=alt.value(rect[1]),
            size=alt.value(15),
            color=alt.value('black')
        )
        return (background+foreground+rule_rect+rule_point_1+rule_point_2+rule_text_1+rule_text_2).properties(
            width=1200,
            height=800
        ).add_selection(selection)
    else:
        return (background+foreground).properties(
            width=1200, 
            height=800
        ).add_selection(selection)

In [2]:
# Read the raw data
df = pd.read_csv('jhu-daily-reports.csv')
df['Active']  = df.Confirmed - (df.Deaths + df.Recovered)
samples = df[['Date', 'Country']].groupby('Date').Country.nunique()
days = samples[samples > 1].index.tolist()
df = df[df['Date'].isin(days)]

/Users/ratreya/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# Global Data Visualizations

In [3]:
from ipywidgets import interact

# Aggregate at country level
country_level = df.groupby(['Country', 'Date'], as_index=False).sum()

# Drop 03-22-2020 since county breakdown started from here and so there is break in continuity
country_level = country_level.drop(country_level[country_level['Date'] == '03-22-2020'].index)
country_level = country_level.drop(country_level[country_level['Date'] == '03-12-2020'].index)
country_level = country_level.drop(country_level[country_level['Date'] == '02-12-2020'].index)

@interact(value=['Active', 'Confirmed', 'Deaths', 'Recovered', 'Confirmed_New', 'Deaths_New', 'Recovered_New'], sort=['Active', 'Confirmed', 'Deaths', 'Recovered', 'Confirmed_New', 'Deaths_New', 'Recovered_New'], limit=(0,50,1))
def chart(value='Confirmed_New', sort='Confirmed_New', limit=10):
    return streamgraph(country_level, 'Country', value, sort, limit).interactive()#.properties(width=608, height=380).save('./output/main.png', scale_factor=2.0)

interactive(children=(Dropdown(description='value', index=4, options=('Active', 'Confirmed', 'Deaths', 'Recove…

In [4]:
country_level = country_level[country_level['Country'].isin(country_level[country_level['Confirmed'] > 1000]['Country'].unique())]
countries = country_level[country_level['Confirmed'] > 1000]['Country'].unique().tolist()

@interact(value=['Active', 'Confirmed', 'Deaths', 'Recovered', 'Confirmed_New', 'Deaths_New', 'Recovered_New'], sort=['Active', 'Confirmed', 'Deaths', 'Recovered', 'Confirmed_New', 'Deaths_New', 'Recovered_New'], limit=(1,50,1), highlight=countries, start=reversed(days), end=reversed(days))
def chart(value='Confirmed_New', sort='Confirmed_New', limit=10, highlight='India', start=None, end=None, xlog=True, ylog=True):
    rect = (start, end)
    return nontimegraph(country_level, 'Country', value, sort, limit, highlight=highlight, xlog=xlog, ylog=ylog, rect=rect).interactive()

interactive(children=(Dropdown(description='value', index=4, options=('Active', 'Confirmed', 'Deaths', 'Recove…

# State Level Visualizations

In [5]:
countries = df[(df['State'].notnull()) & (df['Confirmed'] > 1000)]['Country'].unique()

def state_data(country):
    state_level = df[df['Country'] == country].groupby(['State', 'Date'], as_index=False).sum()
    state_level['Active'] = state_level.Confirmed - (state_level.Deaths + state_level.Recovered)
    if country == 'US':
        # Drop 03-22-2020 since county breakdown started from here and so there is break in continuity
        state_level = state_level.drop(state_level[state_level['Date'] == '03-22-2020'].index)
        state_level = state_level.drop(state_level[state_level['Date'] == '03-18-2020'].index)
        state_level = state_level.drop(state_level[state_level['Date'] == '08-31-2020'].index)
        state_level = state_level[state_level['Date'] >= '03-01-2020']
    if country == 'India':
        state_level['State'].replace(['Cases being reassigned to states'], 'Unassigned', inplace=True)
        state_level = state_level[state_level['Date'] >= '03-01-2020']
        state_level = state_level.drop(state_level[state_level['Date'] == '07-28-2020'].index)
    for var in state_level['State'].unique():
        confirmed = state_level[state_level['State'] == var].Confirmed_New.values.tolist()
        confirmed_pcts = [np.nan] + [(confirmed[i] - confirmed[i-1])/confirmed[i-1]*100 if confirmed[i-1] > 0 else np.nan for i in range(1, len(confirmed))]
        state_level.loc[state_level['State'] == var, 'Confirmed_Pct'] = confirmed_pcts
    return state_level

@interact(country=countries, value=['Active', 'Confirmed', 'Deaths', 'Recovered', 'Confirmed_New', 'Deaths_New', 'Recovered_New'], sort=['Active', 'Confirmed', 'Deaths', 'Recovered', 'Confirmed_New', 'Deaths_New', 'Recovered_New'], limit=(0,50,1))
def chart(country='US', value='Confirmed_New', sort='Confirmed_New', limit=10):
    graph = streamgraph(state_data(country), 'State', value, sort, limit)
    graph.save('us-by-state.html')
    return graph.interactive()#.properties(width=608, height=380).save('./output/us-confirmed-new.png', scale_factor=3.0)

interactive(children=(Dropdown(description='country', index=21, options=('Australia', 'Belgium', 'Brazil', 'Ca…

In [6]:
@interact(country=countries, value=['Active', 'Confirmed', 'Deaths', 'Recovered', 'Confirmed_New', 'Deaths_New', 'Recovered_New'], sort=['Active', 'Confirmed', 'Deaths', 'Recovered', 'Confirmed_New', 'Confirmed_Pct', 'Deaths_New', 'Recovered_New'], limit=(1,50,1))
def chart(country='US', value='Confirmed_New', sort='Confirmed_New', limit=10, xlog=True, ylog=True):
    return nontimegraph(state_data(country), 'State', value, sort, limit, xlog=xlog, ylog=ylog).interactive()#.properties(width=608, height=380).save('./output/india-confirmed-new.png', scale_factor=3.0)

interactive(children=(Dropdown(description='country', index=21, options=('Australia', 'Belgium', 'Brazil', 'Ca…

# US County Level Visualizations

In [7]:
us_state_level = state_data('US')
states=us_state_level[us_state_level['Confirmed'] > 1000]['State'].unique()
@interact(value=['Active', 'Confirmed', 'Deaths', 'Recovered', 'Confirmed_New', 'Deaths_New', 'Recovered_New'], sort=['Active', 'Confirmed', 'Deaths', 'Recovered', 'Confirmed_New', 'Deaths_New', 'Recovered_New'], limit=(0,50,1), state=states)
def chart(state='CA', value='Confirmed_New', sort='Confirmed_New', limit=10):
    county_level = df[df['State'] == state].fillna({'County': state}).groupby(['County', 'Date'], as_index=False).sum()
    return streamgraph(county_level, 'County', value, sort, limit).interactive()#.properties(width=608, height=380).save('WA.png', scale_factor=3.0)

interactive(children=(Dropdown(description='state', index=4, options=('AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT'…

In [8]:
states=us_state_level[us_state_level['Confirmed'] > 1000]['State'].unique()

@interact(value=['Active', 'Confirmed', 'Deaths', 'Recovered', 'Confirmed_New', 'Deaths_New', 'Recovered_New'], sort=['Active', 'Confirmed', 'Deaths', 'Recovered', 'Confirmed_New', 'Deaths_New', 'Recovered_New'], limit=(1,50,1), state=states)
def chart(state='CA', value='Confirmed_New', sort='Confirmed_New', limit=10, xlog=True, ylog=True):
    county_level = df[df['State'] == state].fillna({'County': state}).groupby(['County', 'Date'], as_index=False).sum()
    return nontimegraph(county_level, 'County', value, sort, limit, xlog=xlog, ylog=ylog).interactive()#.properties(width=608, height=380).save('CA.png', scale_factor=3.0)

interactive(children=(Dropdown(description='state', index=4, options=('AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT'…

In [9]:
@interact(value=['Active', 'Confirmed', 'Deaths', 'Recovered', 'Confirmed_New', 'Deaths_New', 'Recovered_New'], sort=['Active', 'Confirmed', 'Deaths', 'Recovered', 'Confirmed_New', 'Deaths_New', 'Recovered_New'], limit=(1,50,1), state = us_state_level['State'].unique())
def chart(value='Confirmed_New', sort='Confirmed_New', limit=10, xlog=True, ylog=True):
    df['Area'] = df.County + ', ' + df.State
    county_level = df.groupby(['Area', 'Date'], as_index=False).sum()

    return nontimegraph(county_level, 'Area', value, sort, limit, xlog=xlog, ylog=ylog).interactive()

interactive(children=(Dropdown(description='value', index=4, options=('Active', 'Confirmed', 'Deaths', 'Recove…

# Cross Country Comparisons

In [10]:
@interact(country=countries, value=['Active', 'Confirmed', 'Deaths', 'Recovered', 'Confirmed_New', 'Deaths_New', 'Recovered_New'], sort=['Active', 'Confirmed', 'Deaths', 'Recovered', 'Confirmed_New', 'Deaths_New', 'Recovered_New', 'Slope'], limit=(1,50,1))
def chart(country='India', value='Confirmed_New', sort='Confirmed_New', limit=10, xlog=True, ylog=True):
    data = state_data(country)
    us = state_data('US')
    data = pd.concat([data, us[us['State'] == 'NY']])
    return nontimegraph(data, 'State', value, sort, limit, xlog=xlog, ylog=ylog).interactive()

interactive(children=(Dropdown(description='country', index=10, options=('Australia', 'Belgium', 'Brazil', 'Ca…

# Hospitalizations

In [11]:
dfh = pd.read_csv('https://covidtracking.com/api/v1/states/daily.csv')
dfh.date = pd.to_datetime(dfh.date, format='%Y%m%d')
dfh.date = dfh.date.dt.strftime('%m-%d-%Y')
dfh = dfh.rename({'date': 'Date', 'state':'State', 'hospitalizedCurrently': 'Hospitalized'}, axis=1)
dfh['Hospitalized'].fillna(0, inplace=True)
data = state_data('US')
data = data.merge(dfh, on=['Date', 'State'], how='outer')

for var in data['State'].unique():
    hospitalized = data[data['State'] == var].Hospitalized.values.tolist()
    hospitalized_deltas = [np.nan] + [hospitalized[i] - hospitalized[i-1] for i in range(1, len(hospitalized))]
    data.loc[data['State'] == var, 'Hospitalized_New'] = hospitalized_deltas
    hospitalized_pcts = [np.nan] + [(hospitalized[i] - hospitalized[i-1])/hospitalized[i-1]*100 if hospitalized[i-1] > 0 else np.nan for i in range(1, len(hospitalized))]
    data.loc[data['State'] == var, 'Hospitalized_Pct'] = hospitalized_pcts

@interact(sort=['Active', 'Confirmed', 'Deaths', 'Recovered', 'Confirmed_New', 'Deaths_New', 'Recovered_New', 'Hospitalized', 'Hospitalized_New', 'Hospitalized_Pct'], limit=(1,50,1))
def chart(sort='Hospitalized_New', limit=10, xlog=True, ylog=True):
    return nontimegraph(data, 'State', 'Hospitalized', sort, limit, addon=['NY'], xlog=xlog, ylog=ylog).interactive()#.properties(width=608, height=380).save('hospitalizations.png', scale_factor=3.0)

interactive(children=(Dropdown(description='sort', index=8, options=('Active', 'Confirmed', 'Deaths', 'Recover…

In [12]:
totals = data.groupby('Date', as_index=False).sum()[['Date', 'Hospitalized']]
alt.Chart(totals).mark_line().encode(
    x='Date:T',
    y='Hospitalized:Q'
).properties(width=608, height=380).interactive()

alt.Chart(...)

# Record Setters

In [13]:
cl = country_level
cl = cl[cl['Date'] == days[-1]].merge(cl.groupby(['Country'], as_index=False).max()[['Country', 'Confirmed_New']], on='Country')
print('Countries settings new highs:')
print(cl[(cl.Confirmed_New_x == cl.Confirmed_New_y) & (cl.Confirmed_New_x != 0)].Country.values)

sl = state_data('US')
sl = sl[sl['Date'] == days[-1]].merge(sl.groupby(['State'], as_index=False).max()[['State', 'Confirmed_New']], on='State')
print('States settings new highs:')
print(sl[(sl.Confirmed_New_x == sl.Confirmed_New_y) & (sl.Confirmed_New_x != 0)].State.values)

cl = country_level
cl = cl[cl['Date'] == days[-1]].merge(cl[cl['Date']>'04-01-2020'].groupby(['Country'], as_index=False).min()[['Country', 'Confirmed_New']], on='Country')
print('Countries settings new lows since April:')
print(cl[(cl.Confirmed_New_x == cl.Confirmed_New_y) & (cl.Confirmed_New_x != 0)].Country.values)

sl = state_data('US')
sl = sl[sl['Date'] == days[-1]].merge(sl[sl['Date']>'04-01-2020'].groupby(['State'], as_index=False).min()[['State', 'Confirmed_New']], on='State')
print('States settings new lows since April:')
print(sl[(sl.Confirmed_New_x == sl.Confirmed_New_y) & (sl.Confirmed_New_x != 0)].State.values)

Countries settings new highs:
['Azerbaijan' 'Belarus' 'Croatia' 'Finland' 'Georgia' 'Indonesia' 'Iran'
 'Kenya' 'Lithuania' 'Russia' 'Serbia' 'Turkey' 'Ukraine' 'Uruguay'
 'West Bank and Gaza']
States settings new highs:
['FL' 'ID' 'KY' 'MI' 'NC' 'OH' 'OR' 'PA' 'SD' 'UT' 'VT']
Countries settings new lows since April:
[]
States settings new lows since April:
[]
